# Example - Reading and Writing with Dask

In [ ]:
import multiprocessing
# Linux/OSX:
import multiprocessing.popen_spawn_posix
# Windows:
# import multiprocessing.popen_spawn_win32
import threading

from dask.distributed import Client, LocalCluster, Lock
from dask.utils import SerializableLock

import rioxarray

%matplotlib inline

When using dask for writing, you should be careful about what lock you use for your process. You need to ensure that
you have a lock for each worker, so the more fine-grained the better.

See docs for:

- [rioxarray.open_rasterio](../rioxarray.rst#rioxarray-open-rasterio)
- DataArray: [rio.to_raster()](../rioxarray.rst#rioxarray.raster_array.RasterArray.to_raster)
- Dataset: [rio.to_raster()](../rioxarray.rst#rioxarray.raster_dataset.RasterDataset.to_raster)

### No distrubuted computing example
Note: Without a lock provided, `to_raster` does not use dask to write to disk.

In [ ]:
xds = rioxarray.open_rasterio(
    "../../test/test_data/compare/small_dem_3m_merged.tif",
    chunks=True,
)
xds.rio.to_raster("simple_write.tif", tiled=True)

### Multithreaded example

In [ ]:
xds = rioxarray.open_rasterio(
    "../../test/test_data/compare/small_dem_3m_merged.tif",
    chunks=True,
)
xds.rio.to_raster("dask_thread.tif", tiled=True, lock=threading.Lock())

### Multiprocessing example

In [ ]:
with LocalCluster() as cluster, Client(cluster) as client:
    mp_manager = multiprocessing.Manager()
    xds = rioxarray.open_rasterio(
        "../../test/test_data/compare/small_dem_3m_merged.tif",
        chunks=True,
    ).persist()
    xds.rio.to_raster(
        "dask_multiprocess.tif", tiled=True, lock=mp_manager.Lock(),
    )

### Multiple worker example

In [ ]:
with LocalCluster() as cluster, Client(cluster) as client:
    xds = rioxarray.open_rasterio(
        "../../test/test_data/compare/small_dem_3m_merged.tif",
        chunks=True,
    ).persist()
    xds.rio.to_raster(
        "dask_multiworker.tif", tiled=True, lock=Lock("rio", client=client),
    )